## Install and Import

In [ ]:
%pip install -q rdkit pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from rdkit import Chem, RDLogger
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
import numpy as np
import tensorflow as tf
from tensorflow import keras

RDLogger.DisableLog("rdApp.*")

## Dataset

In [ ]:
class DataHelper():
    def __init__(self):
        csv_path = tf.keras.utils.get_file("qm9.csv", "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")
        self.data = list()
        with open(csv_path, "r")  as fin:
            for line in fin.readlines()[1:]:
                self.data.append(line.split(",")[1])

    def __getitem__(self, idx: int):
        smiles = self.data[idx]
        print(f"SMILES: {smiles}")
        mol = Chem.MolFromSmiles(smiles)
        print(f"Number of Heavy Atoms: {mol.GetNumHeavyAtoms()}")
        return mol

In [ ]:
DH = DataHelper()

29856825/29856825 [==============================] - 1s 0us/step


## Utilities

In [ ]:
atom_mapping = {
    "C": 0,
    0: "C",
    "N": 1,
    1: "N",
    "O": 2,
    2: "O",
    "F": 3,
    3: "F",
}

bond_mapping = {
    "SINGLE": 0,
    0: Chem.BondType.SINGLE,
    "DOUBLE": 1,
    1: Chem.BondType.DOUBLE,
    "TRIPLE": 2,
    2: Chem.BondType.TRIPLE,
    "AROMATIC": 3,
    3: Chem.BondType.AROMATIC,
}

NUM_ATOMS = 9  # Maximum number of atoms
ATOM_DIM = 4 + 1  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 64  # Size of the latent space


def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # loop over each atom in molecule
    for atom in molecule.GetAtoms():
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]
        # loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type_idx = bond_mapping[bond.GetBondType().name]
            adjacency[bond_type_idx, [i, j], [j, i]] = 1

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule

## Generate Training Set

In [ ]:
adjacency_tensor, feature_tensor = [], []
for smiles in DH.data[::10]:
    adjacency, features = smiles_to_graph(smiles)
    adjacency_tensor.append(adjacency)
    feature_tensor.append(features)

adjacency_tensor = np.array(adjacency_tensor)
feature_tensor = np.array(feature_tensor)

print("adjacency_tensor.shape =", adjacency_tensor.shape)
print("feature_tensor.shape =", feature_tensor.shape)

adjacency_tensor.shape = (13389, 5, 9, 9)
feature_tensor.shape = (13389, 9, 5)


## Old Generator

### Quantum LSTM Cell

In [ ]:
%pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 30.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.1
    Uninstalling autograd-1.6.1:
      Successfully uninstalled autograd-1.6.1


In [ ]:
import pennylane as qml

In [ ]:
class QLSTMCell(keras.Model):
    def __init__(self, input_size, hidden_size, n_qubits, n_layers=1, backend="default.qubit"):
        super(QLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.backend = backend

        self.wires_forget = [f"wire_forget_{i}" for i in range(self.n_qubits)]
        self.wires_inputs = [f"wire_inputs_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_forget = qml.device(self.backend, wires=self.wires_forget)
        self.dev_inputs = qml.device(self.backend, wires=self.wires_inputs)
        self.dev_update = qml.device(self.backend, wires=self.wires_update)
        self.dev_output = qml.device(self.backend, wires=self.wires_output)

        def _circuit_forget(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_forget)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_forget)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_forget]
        self.qlayer_forget = qml.QNode(_circuit_forget, self.dev_forget, interface="tf")

        def _circuit_input(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_inputs)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_inputs, rotation=qml.RY)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_inputs]
        self.qlayer_inputs = qml.QNode(_circuit_input, self.dev_inputs, interface="tf")

        def _circuit_update(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_update)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_update)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_update]
        self.qlayer_update = qml.QNode(_circuit_update, self.dev_update, interface="tf")

        def _circuit_output(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_output)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_output)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_output]
        self.qlayer_output = qml.QNode(_circuit_output, self.dev_output, interface="tf")

        weight_shapes = {"weights": (n_layers, n_qubits)}

        self.cell = keras.layers.Dense(input_size + hidden_size, use_bias=True)
        # default args = xavier_uniform for weight and zeros for bias

        self.VQC = {
            'forget': qml.qnn.KerasLayer(self.qlayer_forget, weight_shapes, n_qubits),
            'inputs': qml.qnn.KerasLayer(self.qlayer_inputs, weight_shapes, n_qubits),
            'update': qml.qnn.KerasLayer(self.qlayer_update, weight_shapes, n_qubits),
            'output': qml.qnn.KerasLayer(self.qlayer_output, weight_shapes, n_qubits)
        }

        self.clayer_out = keras.layers.Dense(n_qubits, use_bias=False)

    def call(self, x, hidden):
        hx, cx = hidden
        gates = tf.concat([x, hx], axis=1)
        gates = self.cell(gates)

        for layer in range(self.n_layers):
            ingate = keras.activations.sigmoid(self.clayer_out(self.VQC['forget'](gates)))
            forgetgate = keras.activations.sigmoid(self.clayer_out(self.VQC['inputs'](gates)))
            cellgate = keras.activations.tanh(self.clayer_out(self.VQC['update'](gates)))
            outgate = keras.activations.sigmoid(self.clayer_out(self.VQC['forget'](gates)))

            cy = tf.math.multiply(cx, forgetgate) + tf.math.multiply(ingate, cellgate)
            hy = tf.math.multiply(outgate, keras.activations.tanh(cy))

        return (hy, cy)


In [ ]:
QL = QLSTMCell(8, 8, 16)

In [ ]:
QL([[152], [191]], [[[14003], [3356]], [[16025], [3332]]])

(<tf.Tensor: shape=(2, 16), dtype=float32, numpy=
 array([[0.5310484 , 0.46706748, 0.49263352, 0.48885173, 0.46100816,
         0.48616278, 0.46705902, 0.5164377 , 0.50630206, 0.47582075,
         0.50538546, 0.45360574, 0.45882186, 0.48910308, 0.49444488,
         0.52046955],
        [0.50003356, 0.4988767 , 0.500993  , 0.49774396, 0.5013281 ,
         0.49914947, 0.50132024, 0.4987475 , 0.5002066 , 0.49794322,
         0.49889502, 0.49887583, 0.5022319 , 0.5010258 , 0.4954545 ,
         0.49760774]], dtype=float32)>,
 <tf.Tensor: shape=(2, 16), dtype=float32, numpy=
 array([[8242.785 , 7791.6787, 7980.9077, 7910.351 , 7759.1963, 7918.0483,
         7782.5093, 8119.2095, 8084.0063, 7842.506 , 8047.505 , 7697.7476,
         7730.1973, 7930.4746, 7933.3203, 8135.8555],
        [1672.5239, 1640.8035, 1681.1849, 1625.6139, 1688.0941, 1659.0619,
         1676.587 , 1642.1991, 1669.8347, 1625.886 , 1653.4589, 1655.7776,
         1700.914 , 1681.2349, 1590.5375, 1625.8953]], dtype=float32)>

In [ ]:
QL.summary()

Model: "qlstm_cell"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             multiple                  48        
                                                                 
 keras_layer (KerasLayer)    multiple                  16        
                                                                 
 keras_layer_1 (KerasLayer)  multiple                  16        
                                                                 
 keras_layer_3 (KerasLayer)  multiple                  16        
                                                                 
 keras_layer_2 (KerasLayer)  multiple                  16        
                                                                 
 dense_4 (Dense)             multiple                  256       
                                                                 
Total params: 368
Trainable params: 368
Non-trainable pa

### Old QLSTM Generator - VERY BUGGY

In [ ]:
class GraphGenerator(keras.Model):
    def __init__(self, H_inputs, H, z_dim, N, rw_len, temp):
        '''
            H_inputs: input dimension
            H:        hidden dimension
            z_dim:    latent dimension
            N:        number of nodes (needed for the up and down projection)
            rw_len:   number of LSTM cells
            temp:     temperature for the gumbel softmax
        '''
        super(GraphGenerator, self).__init__()
        self.intermediate = keras.layers.Dense(H)
        self.c_up = keras.layers.Dense(H)
        self.h_up = keras.layers.Dense(H)
        self.qlstm = QLSTMCell(input_size=H_inputs, hidden_size=H, n_qubits=N)
        self.W_up = keras.layers.Dense(N)
        self.W_down = keras.layers.Dense(H_inputs, use_bias=False)
        self.rw_len = rw_len
        self.temp = temp
        self.H, self.N = H, N
        self.H_inputs = H_inputs
        self.latent_dim = z_dim

    def sample_latent(self, num_samples):
        return tf.random.normal([num_samples, self.latent_dim])

    # I HAVE NO IDEA HOW TO DO THIS IN TENSORFLOW
    def init_hidden(self, batch_sz):
        weight = next(self.parameters).data
        return weight.new(batch_sz, self.H_inputs).zero_()
    # HELP WITH FUNCTION ABOVE

    def sample_gumbel(self, logits, eps=1e-20):
        U = tf.random.normal([logits.shape])
        return -tf.math.log(-tf.math.log(U + eps) + eps)

    def gumbel_softmax(self, logits, temp):
        gumbel = self.sample_gumbel(logits)
        y = logits + gumbel
        y = tf.nn.softmax(y / temp, axis=1)
        return y

    def call(self, latent, inputs):
        intermediate = keras.activations.tanh(self.intermediate(latent))
        hc = (
            keras.activations.tanh(self.h_up(intermediate)),
            keras.activations.tanh(self.c_up(intermediate))
        )
        out = []
        for i in range(self.rw_len):
            hh, cc = self.qlstm(inputs, hc)
            hc = (hh, cc)
            h_up = self.W_up(hh)
            h_sample = self.gumbel_softmax(h_up, self.temp)
            inputs = self.W_down(h_sample)
            out.append(h_sample)
        return tf.stack(out, axis=1)

    def sample_reg(self, num_samples):
        noise = self.sample_latent(num_samples)
        inp_zeroes = self.init_hidden(num_samples)
        gen_data = self(noise, inp_zeroes)
        return gen_data

    #Not sure if this works
    def sample_disc(self, num_samples):
        proba = tf.stop_gradient(self.sample_reg(num_samples))
        return np.argmax(proba.numpy(), axis=2)


In [ ]:
QG = GraphGenerator(16, 16, 16, 16, 1, 0.5)

In [ ]:
QG([314.1592653], [7438, 9465])

ValueError: ignored

## New Generator

### Quantum Dense Layer

#### Extra Utilities/Imports

In [ ]:
%pip install -q qiskit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.4 MB/s eta 0:00:00


In [ ]:
import qiskit

In [ ]:
from qiskit import transpile, assemble, QuantumRegister, QuantumCircuit
from qiskit.providers.ibmq import least_busy
from qiskit.providers.ibmq.job import job_monitor
from qiskit.tools import backend_monitor

#### IDK what these functions do

In [ ]:
class QiskitCircuitModule:
    def __init__(self, qubits, instructions=None, execute_on_IBMQ=False, shots=2):
        self.qubit_num = qubits
        self.instructions = instructions
        if not self.instructions:
            self.instructions = self.null_circuit(self.qubit_num)
        self.probabilities = tf.constant([[0.5] * self.qubit_num])
        self.phase_probabilities = tf.constant([1] * self.qubit_num)
        self.layer = self.superposition_qubits(self.probabilities, self.phase_probabilities)
        self.layer.append(self.instructions, range(self.qubit_num))
        self.layer.measure_all()
        self.backend = qiskit.Aer.get_backend('aer_simulator')

    def p_to_angle(self, p):
        angle = 2 * np.arccos(np.sqrt(p))
        return angle

    def superposition_qubits(self, probabilities: tf.Tensor, phases: tf.Tensor):
        layer = qiskit.QuantumCircuit(self.qubit_num)
        reshaped_probabilities = tf.reshape(probabilities, [self.qubit_num])
        reshaped_phases = tf.reshape(phases, [self.qubit_num])
        static_probabilities = tf.get_static_value(reshaped_probabilities[:])
        static_phases = tf.get_static_value(reshaped_phases[:])
        try:
            for i in range(len(static_probabilities)):
                p = static_probabilities[i]
            #for ix, p in enumerate(static_probabilities):
                p = np.abs(p)
                theta = self.p_to_angle(p)
                phi = self.p_to_angle(static_phases[i])
                layer.u(theta, phi, 0, i)
        except TypeError:
            print(reshaped_probabilities)
            raise RuntimeError("Debugging!")
        return layer

    def quantum_execute(self, probabilities, phases):
        self.layer = self.superposition_qubits(probabilities, phases)
        self.layer.append(self.instructions, range(self.qubit_num))
        self.layer.measure_all()
        transpiled_circuit = transpile(self.layer, self.backend)
        quantum_job_object = assemble(transpiled_circuit, shots=self.shots)
        quantum_job = self.backend.run(quantum_job_object)
        result = quantum_job.result().get_counts()
        qubit_set_probabilities = self.calculate_qubit_set_probabilities(result)
        return qubit_set_probabilities

    def calculate_qubit_set_probabilities(self, quantum_job_result):
        qubit_set_probabilities = [0] * self.qubit_num
        for state_result, count in quantum_job_result.items():
            for ix, q in enumerate(state_result):
                if q == '1':
                    qubit_set_probabilities[ix] += count
        sum_counts = sum(qubit_set_probabilities)
        if not sum_counts == 0:
            qubit_set_probabilities = [i/sum_counts for i in qubit_set_probabilities]
        return qubit_set_probabilities

    def null_circuit(self, qubits):
        try:
            gate_register = QuantumRegister(qubits, 'q')
            gate_circuit = QuantumCircuit(gate_register, name='sub_circuit')
            gate_instructions = gate_circuit.to_instruction()
        except Exception as e: raise RuntimeError("🅵🆄🅲🅺!")
        return gate_instructions

#### Quantum Dense Layer

In [ ]:
class QuantumLayer(keras.layers.Layer):
    def __init__(self, qubits=16, instructions=None, shots=2, use_parameter_shift_gradient_flow=False):
        super(QuantumLayer, self).__init__()
        self.use_parameter_shift_gradient_flow = use_parameter_shift_gradient_flow
        self.qubits = qubits
        self.instructions = instructions
        self.tensor_history = []
        self.shots = shots
        self.circuit = QiskitCircuitModule(self.qubits, instructions=self.instructions, shots=self.shots)

    def build(self, input_shape):
        kernel_p_initialisation = tf.random_normal_initializer()
        self.kernel_p = tf.Variable(name="kernel_p", initial_value=kernel_p_initialisation(shape=(input_shape[-1], self.qubits), dtype='float32'), trainable=True)
        kernel_phi_initialisation = tf.zeros_initializer()
        self.kernel_phi = tf.Variable(name="kernel_phi", initial_value=kernel_phi_initialisation(shape=(self.qubits,), dtype='float32'), trainable=False)

    def call(self, inputs):
        if not self.use_parameter_shift_gradient_flow:
            output = tf.matmul(inputs, self.kernel_p)
            qubit_output = self.circuit.quantum_execute(tf.reshape(output, [1, self.qubits]), self.kernel_phi)
            qubit_output = tf.reshape(tf.convert_to_tensor(qubit_output), (1, 1, self.qubits))
            output += (qubit_output - output)
        else: output = self.quantum_flow(inputs)
        return output

    @tf.custom_gradient
    def quantum_flow(self, x):
        output = tf.matmul(x, self.kernel_p)
        qubit_output = tf.reshape(tf.convert_to_tensor(self.circuit.quantum_execute(tf.reshape(output, [1, self.qubits]), self.kernel_phi)), (1, 1, self.qubits))
        output = qubit_output

        def grad(dy, variables=None):
            shift = np.pi / 2
            shift_right = x + np.ones(x.shape) * shift
            shift_left = x - np.ones(x.shape) * shift
            input_left = tf.matmul(shift_left, self.kernel_p)
            input_right = tf.matmul(shift_right, self.kernel_p)
            output_right = self.circuit.quantum_execute(tf.reshape(input_right, [1, self.qubits]), self.kernel_phi)
            output_left = self.circuit.quantum_execute(tf.reshape(input_left, [1, self.qubits]), self.kernel_phi)
            quantum_gradient = [output_right[i] - output_left[i] for i in range(len(output_right))]
            input_gradient = dy * quantum_gradient
            dy_input_gradient = tf.reshape(tf.matmul(input_gradient, tf.transpose(self.kernel_p)), shape=[1, 1, x.get_shape().as_list()[-1]])
            grd_w = []
            for i in range(self.qubits):
                w = self.kernel_p[:, i]
                w += dy_input_gradient
                grd_w.append(w)
            tf_grd_w = tf.convert_to_tensor(grd_w)
            tf_grd_w = tf.reshape(tf_grd_w, shape=(x.get_shape().as_list()[-1], self.qubits))
            return dy_input_gradient, [tf_grd_w]

        return output, grad

#### Please for the love of Allah please work

In [ ]:
def AllahuAkbar(dense_units, dropout_rate, latent_dim, adjacency_shape, feature_shape, param_shift=False,):
        z = keras.layers.Input(shape=(64,))
        driver = keras.layers.Dense(16, activation="relu")
        x = driver(z)
        for units in dense_units:
            x = QuantumLayer(units, use_parameter_shift_gradient_flow=param_shift)(x)
            x = keras.activations.tanh()(x)
            x = keras.layers.Dropout(dropout_rate)(x)
        x_adjacency = keras.layers.Dense(tf.math.reduce_prod(adjacency_shape))(x)
        x_adjacency = keras.layers.Reshape(adjacency_shape)(x_adjacency)
        x_adjacency = (x_adjacency + tf.transpose(x_adjacency, (0, 1, 3, 2))) / 2
        x_adjacency = keras.layers.Softmax(axis=1)(x_adjacency)
        x_features = keras.layers.Dense(tf.math.reduce_prod(feature_shape))(x)
        x_features = keras.layers.Reshape(feature_shape)(x_features)
        x_features = keras.layers.Softmax(axis=2)(x_features)
        return keras.Model(inputs=z, outputs=[x_adjacency, x_features], name="Allah")

In [ ]:
allah = AllahuAkbar(
    dense_units=[4, 8, 16],
    dropout_rate=0.2,
    latent_dim=LATENT_DIM,
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM),
)

In [ ]:
asdfg = tf.Tensor("quantum_layer_13/Reshape_1:0", shape=(4,), dtype=float32)

NameError: ignored

### Default Tutorial Generator - NOT QUANTUM YET

In [ ]:
def GraphGenerator(dense_units, dropout_rate, latent_dim, adjacency_shape, feature_shape,):
    z = keras.layers.Input(shape=(64,))
    x = z
    for units in dense_units:
        x = keras.layers.Dense(units, activation="tanh")(x)
        x = keras.layers.Dropout(dropout_rate)(x)
    x_adjacency = keras.layers.Dense(tf.math.reduce_prod(adjacency_shape))(x)
    x_adjacency = keras.layers.Reshape(adjacency_shape)(x_adjacency)
    x_adjacency = (x_adjacency + tf.transpose(x_adjacency, (0, 1, 3, 2))) / 2
    x_adjacency = keras.layers.Softmax(axis=1)(x_adjacency)
    x_features = keras.layers.Dense(tf.math.reduce_prod(feature_shape))(x)
    x_features = keras.layers.Reshape(feature_shape)(x_features)
    x_features = keras.layers.Softmax(axis=2)(x_features)
    return keras.Model(inputs=z, outputs=[x_adjacency, x_features], name="Generator")

In [ ]:
generator = GraphGenerator(
    dense_units=[128, 256, 512],
    dropout_rate=0.2,
    latent_dim=LATENT_DIM,
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM),
)

In [ ]:
generator.summary()

Model: "Generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 128)          8320        ['input_3[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 128)          0           ['dense_5[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 256)          33024       ['dropout_2[0][0]']              
                                                                                          

## Graph Discriminator

### Relational Graph Convolution Layer

In [ ]:
class RelationalGraphConvLayer(keras.layers.Layer):
    def __init__(
        self,
        units=128,
        activation="relu",
        use_bias=False,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.units = units
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)

    def build(self, input_shape):
        bond_dim = input_shape[0][1]
        atom_dim = input_shape[1][2]

        self.kernel = self.add_weight(
            shape=(bond_dim, atom_dim, self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            trainable=True,
            name="W",
            dtype=tf.float32,
        )

        if self.use_bias:
            self.bias = self.add_weight(
                shape=(bond_dim, 1, self.units),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                trainable=True,
                name="b",
                dtype=tf.float32,
            )

        self.built = True

    def call(self, inputs, training=False):
        adjacency, features = inputs
        # Aggregate information from neighbors
        x = tf.matmul(adjacency, features[:, None, :, :])
        # Apply linear transformation
        x = tf.matmul(x, self.kernel)
        if self.use_bias:
            x += self.bias
        # Reduce bond types dim
        x_reduced = tf.reduce_sum(x, axis=1)
        # Apply non-linear transformation
        return self.activation(x_reduced)

### Actual Disciminator

In [ ]:
def GraphDiscriminator(
    gconv_units, dense_units, dropout_rate, adjacency_shape, feature_shape
):

    adjacency = keras.layers.Input(shape=adjacency_shape)
    features = keras.layers.Input(shape=feature_shape)

    # Propagate through one or more graph convolutional layers
    features_transformed = features
    for units in gconv_units:
        features_transformed = RelationalGraphConvLayer(units)(
            [adjacency, features_transformed]
        )

    # Reduce 2-D representation of molecule to 1-D
    x = keras.layers.GlobalAveragePooling1D()(features_transformed)

    # Propagate through one or more densely connected layers
    for units in dense_units:
        x = keras.layers.Dense(units, activation="relu")(x)
        x = keras.layers.Dropout(dropout_rate)(x)

    # For each molecule, output a single scalar value expressing the
    # "realness" of the inputted molecule
    x_out = keras.layers.Dense(1, dtype="float32")(x)

    return keras.Model(inputs=[adjacency, features], outputs=x_out)

In [ ]:
NUM_ATOMS = 9  # Maximum number of atoms
ATOM_DIM = 4 + 1  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 64  # Size of the latent space

In [ ]:
discriminator = GraphDiscriminator(
    gconv_units=[128, 128, 128, 128],
    dense_units=[512, 512],
    dropout_rate=0.2,
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM),
)

In [ ]:
discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 9, 9)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 9, 5)]       0           []                               
                                                                                                  
 relational_graph_conv_layer (R  (None, 9, 128)      3200        ['input_1[0][0]',                
 elationalGraphConvLayer)                                         'input_2[0][0]']                
                                                                                                  
 relational_graph_conv_layer_1   (None, 9, 128)      81920       ['input_1[0][0]',            

## Compile Final Model - TODO